# PyTorch로 Transformar 간단 구현

In [3]:
!pip install bertviz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 인코더

##### Self-Attention

In [4]:
# 시각화: 단어와 뉴런 간 연결 분석 시각화(단일 문장)
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

Output hidden; open in https://colab.research.google.com to view.

In [5]:
# 시각화2: 어텐션 헤드 간의 단어 관계 시각화(문장 간 관계 포함)
from transformers import AutoModel
from bertviz import head_view

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sent1 = "time flies like an arrow"
sent2 = "fruit flies like a banana"

viz_inputs = tokenizer(sent1, sent2, return_tensors="pt")
attention = model(**viz_inputs).attentions
sent2_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs["input_ids"][0])

head_view(attention, tokens, sent2_start, heads=[8])

Output hidden; open in https://colab.research.google.com to view.

BERT 모델 (bert-base-uncased) <br>
|---- 12개의 레이어 (Transformaer Layers) <br>
|----|---- 12개의 어텐션 헤드(각 레이어마다) <br>
|----|----|---- 64 차원 백터 출력(각 헤드마다) <br>
|----|----|---- 12개 헤드의 출력을 합쳐서 768차원의 뉴런 백터(hidden_state) 생성

In [6]:
# 토큰화
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [7]:
# Embedding layer 생성
import torch.nn as nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
emb = nn.Embedding(config.vocab_size, config.hidden_size)
emb

Embedding(30522, 768)

In [8]:
# Embedding 처리
inputs_embedded =emb(inputs.input_ids)
inputs_embedded.shape

torch.Size([1, 5, 768])

In [9]:
# Self-Attention 메커니즘
import torch
import torch.nn.functional as F
from math import sqrt


# 1. score 계산
query = key = value = inputs_embedded
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)

# 2. softmax 적용
weights = F.softmax(scores, dim=-1)

# 3. Attention value 계산
attn_outputs = torch.bmm(weights, value)
attn_outputs.shape

torch.Size([1, 5, 768])

In [10]:
# Self-Attention 함수
def attention(query, key, value):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)

In [11]:
# Muti-HeadAttention

class AttentionHead(nn.Module):
  def __init__(self, embed_dim, head_dim):
    super().__init__()
    self.q = nn.Linear(embed_dim, head_dim)
    self.k = nn.Linear(embed_dim, head_dim)
    self.v = nn.Linear(embed_dim, head_dim)

  def forward(self, hidden_attention):
    return attention(self.q(hidden_attention), self.k(hidden_attention), self.v(hidden_attention))


In [12]:
class MultiHeadAttention(nn.Module):
  def __init__(self, config):
    super().__init__()
    embed_dim = config.hidden_size
    heads_num = config.num_attention_heads
    head_dim = embed_dim // heads_num
    self.heads = nn.ModuleList(
        [AttentionHead(embed_dim, head_dim) for _ in range(heads_num)]
    )
    self.output_linear = nn.Linear(embed_dim, embed_dim)

  def forward(self, hidden_state):
    output = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
    output = self.output_linear(inputs_embedded)
    return output

In [13]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embedded)
attn_outputs.size

<function Tensor.size>

##### Feed-Forward

- 간단한 두 개 층으로 구성된 완전연결 신경망
- 전체 임베딩 시퀀스를 하나의 백터로 처리하지 않고 각 임베딩을 독립적으로 관리
- GELU(Gaussian Error Linear Unit)활성 함수 많이 사용

In [17]:
class FeedForward(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
    self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
    self.gelu = nn.GELU()
    self.dropout = nn.Dropout(config.hidden_dropout_prob)


  def forward(self, x):
    x = self.linear_1(x)
    x = self.gelu(x)
    x = self.linear_2(x)
    x = self.dropout(x)
    return x

In [18]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()

torch.Size([1, 5, 768])

##### 정규화를 반영한 Encoder Layer
- 사전 층 정규화

In [23]:
class TransformerEncoderLayer(nn.Module):
  def __init__(self, confing):
    super().__init__()
    self.layer_norm1 = nn.LayerNorm(config.hidden_size)
    self.layer_norm2 = nn.LayerNorm(config.hidden_size)
    self.attention = MultiHeadAttention(config)
    self.feed_forward = FeedForward(config)

  def forward(self, x):
    hidden_state = self.layer_norm1(x)
    x = x + self.attention(hidden_state)
    x = x + self.feed_forward(self.layer_norm2(x))
    return x

In [24]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embedded.shape, encoder_layer(inputs_embedded).shape

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

##### 위치 임베딩(Positional Embedding)

In [27]:
class Embeddings(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.token_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)
    self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
    self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
    self.dropout = nn.Dropout()

  def forward(self, inputs_ids):
    seq_length = inputs_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
    token_embeddings = self.token_embeddings(inputs_ids)
    position_embeddings = self.position_embeddings(position_ids)
    embeddings = token_embeddings + position_embeddings
    embeddings = self.layer_norm(embeddings)
    embeddings = self.dropout(embeddings)
    return embeddings

In [28]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [30]:
class TransformerEncoder(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.embeddings = Embeddings(config)
    self.layers = nn.ModuleList([TransformerEncoderLayer(config) for _ in range(config.num_hidden_layers)])

  def forward(self, x):
    x = self.embedings(x)
    for layer in layers:
      x = layer(x)
    return x

### 디코더

- "Masked" Multi-Head Self-Attention
- Encoder-Decoder Attention

In [34]:
seq_len = inputs.input_ids.size(-1)
mask = torch.tril(torch.ones(seq_len, seq_len)).unsqueeze(0)
mask[0]


tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [35]:
scores.masked_fill(mask == 0, -float("inf"))

tensor([[[25.7532,    -inf,    -inf,    -inf,    -inf],
         [-0.3032, 29.2594,    -inf,    -inf,    -inf],
         [ 0.8864,  1.0849, 29.7168,    -inf,    -inf],
         [ 1.0921,  1.3485,  0.2160, 27.2406,    -inf],
         [ 0.3147, -0.7989, -1.1203, -0.9587, 27.4997]]],
       grad_fn=<MaskedFillBackward0>)

In [ ]:
# attention + masking
def attention(query, key, value, mask=None):
  dim_k = query.size(-1)
  scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
  if mask is not None:
    scores = scores.masked_fill(mask == 0, -float("inf"))
  weights = F.softmax(scores, dim=-1)
  return torch.bmm(weights, value)